In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# foreground_classes = {'plane', 'car', 'bird'}

# background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

# fg1,fg2,fg3 = 0,1,2

In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [7]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0)
    self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0)
    self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.conv7 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.batch_norm1 = nn.BatchNorm2d(32, track_running_stats = False)
    self.batch_norm2 = nn.BatchNorm2d(128, track_running_stats = False)
    self.dropout1 = nn.Dropout2d(p=0.05)
    self.dropout2 = nn.Dropout2d(p=0.1)
    self.fc1 = nn.Linear(128,64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 10)


  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(self.batch_norm1(x))

    x = (F.relu(self.conv2(x)))
    x = self.pool(x)
    
    x = self.conv3(x)
    x = F.relu(self.batch_norm2(x))

    x = (F.relu(self.conv4(x)))
    x = self.pool(x)
    x = self.dropout1(x)

    x = self.conv5(x)
    x = F.relu(self.batch_norm2(x))

    x = (F.relu(self.conv6(x)))
    x = self.pool(x)

    x = self.conv7(x)
    x = F.relu(self.batch_norm2(x))

    x = self.conv8(x)
    x = F.relu(self.batch_norm2(x))
    
    x = x.view(x.size(0), -1)

    x = self.dropout2(x)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.dropout2(x)
    x = self.fc3(x)
    return x

In [8]:
cnn_net = CNN()#.double()
cnn_net = cnn_net.to("cuda")

In [9]:
print(cnn_net)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (dropout1): Dropout2d(p=0.05, inplace=False)
  (dropout2): Dropout2d(p=0.1, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Line

In [10]:
for i,j in cnn_net.state_dict().items():
  print(i)

conv1.weight
conv1.bias
conv2.weight
conv2.bias
conv3.weight
conv3.bias
conv4.weight
conv4.bias
conv5.weight
conv5.bias
conv6.weight
conv6.bias
conv7.weight
conv7.bias
conv8.weight
conv8.bias
batch_norm1.weight
batch_norm1.bias
batch_norm2.weight
batch_norm2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias


In [11]:
for i,j in cnn_net.state_dict().items():
  if i == 'batch_norm1.weight':
    print(i,j)

batch_norm1.weight tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')


In [12]:
cnn_net.load_state_dict(torch.load("/content/drive/My Drive/Research/train_begining_layers_vs_last_layers/"+"cnn_net_8layer"+".pt"))

<All keys matched successfully>

In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 99 %
50000 49880


In [14]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 80 %
10000 8021


In [15]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 75 %
Accuracy of   car : 92 %
Accuracy of  bird : 84 %
Accuracy of   cat : 54 %
Accuracy of  deer : 84 %
Accuracy of   dog : 53 %
Accuracy of  frog : 83 %
Accuracy of horse : 91 %
Accuracy of  ship : 85 %
Accuracy of truck : 76 %


In [16]:
cnn_net.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0).to("cuda")
cnn_net.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0).to("cuda")
cnn_net.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0).to("cuda")
cnn_net.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0).to("cuda")

In [17]:
cnn_net.conv5.weight.requires_grad  = False
cnn_net.conv5.bias.requires_grad  = False
cnn_net.conv6.weight.requires_grad  = False
cnn_net.conv6.bias.requires_grad  = False
cnn_net.conv7.weight.requires_grad  = False
cnn_net.conv7.bias.requires_grad  = False
cnn_net.conv8.weight.requires_grad  = False
cnn_net.conv8.bias.requires_grad  = False
cnn_net.fc1.weight.requires_grad  = False
cnn_net.fc1.bias.requires_grad  = False
cnn_net.fc2.weight.requires_grad  = False
cnn_net.fc2.bias.requires_grad  = False
cnn_net.fc3.weight.requires_grad  = False
cnn_net.fc3.bias.requires_grad  = False

In [18]:
for param in cnn_net.parameters():
  print(param.requires_grad)

True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False


In [19]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 15 %
50000 7915


In [20]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 15 %
10000 1587


In [21]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane :  6 %
Accuracy of   car : 15 %
Accuracy of  bird : 15 %
Accuracy of   cat : 31 %
Accuracy of  deer :  7 %
Accuracy of   dog : 20 %
Accuracy of  frog :  0 %
Accuracy of horse :  0 %
Accuracy of  ship : 28 %
Accuracy of truck : 11 %


In [22]:
import torch.optim as optim
criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.SGD(cnn_net.parameters(), lr=0.01, momentum=0.9)

In [23]:
acti = []
loss_curi = []
epochs = 300
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_cnn.zero_grad()

        # forward + backward + optimize
        outputs = cnn_net(inputs)
        loss = criterion_cnn(outputs, labels)
        loss.backward()
        optimizer_cnn.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.01):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 3.703
[1,   100] loss: 2.685
[1,   150] loss: 2.541
[2,    50] loss: 2.294
[2,   100] loss: 2.175
[2,   150] loss: 2.043
[3,    50] loss: 1.899
[3,   100] loss: 1.810
[3,   150] loss: 1.752
[4,    50] loss: 1.607
[4,   100] loss: 1.582
[4,   150] loss: 1.485
[5,    50] loss: 1.416
[5,   100] loss: 1.333
[5,   150] loss: 1.294
[6,    50] loss: 1.152
[6,   100] loss: 1.126
[6,   150] loss: 1.048
[7,    50] loss: 0.967
[7,   100] loss: 0.940
[7,   150] loss: 0.931
[8,    50] loss: 0.831
[8,   100] loss: 0.819
[8,   150] loss: 0.798
[9,    50] loss: 0.727
[9,   100] loss: 0.703
[9,   150] loss: 0.717
[10,    50] loss: 0.601
[10,   100] loss: 0.618
[10,   150] loss: 0.657
[11,    50] loss: 0.544
[11,   100] loss: 0.561
[11,   150] loss: 0.540
[12,    50] loss: 0.486
[12,   100] loss: 0.506
[12,   150] loss: 0.497
[13,    50] loss: 0.432
[13,   100] loss: 0.442
[13,   150] loss: 0.459
[14,    50] loss: 0.415
[14,   100] loss: 0.407
[14,   150] loss: 0.407
[15,    50] loss: 0

In [24]:
torch.save(cnn_net.state_dict(),"/content/drive/My Drive/Research/train_begining_layers_vs_last_layers/weights"+"CIFAR10_last4layer_fixed_cnn8layer"+".pt")

In [25]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 99 %
50000 49822


In [26]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= cnn_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 77 %
10000 7774


In [27]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        outputs = cnn_net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 68 %
Accuracy of   car : 92 %
Accuracy of  bird : 84 %
Accuracy of   cat : 31 %
Accuracy of  deer : 53 %
Accuracy of   dog : 60 %
Accuracy of  frog : 66 %
Accuracy of horse : 66 %
Accuracy of  ship : 90 %
Accuracy of truck : 88 %
